> # Day 6: Wait For It
> 
> The ferry quickly brings you across Island Island. After asking around, you discover that there is indeed normally a large pile of sand somewhere near here, but you don't see anything besides lots of water and the small island where the ferry has docked.
> 
> As you try to figure out what to do next, you notice a poster on a wall near the ferry dock. "Boat races! Open to the public! Grand prize is an all-expenses-paid trip to Desert Island!" That must be where the sand comes from! Best of all, the boat races are starting in just a few minutes.
> 
> You manage to sign up as a competitor in the boat races just in time. The organizer explains that it's not really a traditional race - instead, you will get a fixed amount of time during which your boat has to travel as far as it can, and you win if your boat goes the farthest.
> 
> As part of signing up, you get a sheet of paper (your puzzle input) that lists the time allowed for each race and also the best distance ever recorded in that race. To guarantee you win the grand prize, you need to make sure you go farther in each race than the current record holder.
> 
> The organizer brings you over to the area where the boat races are held. The boats are much smaller than you expected - they're actually toy boats, each with a big button on top. Holding down the button charges the boat, and releasing the button allows the boat to move. Boats move faster if their button was held longer, but time spent holding the button counts against the total race time. You can only hold the button at the start of the race, and boats don't move until the button is released.
> 
> For example:
>```
Time:      7  15   30
Distance:  9  40  200
>```
> This document describes three races:
> 
>- The first race lasts 7 milliseconds. The record distance in this race is 9 millimeters.
>- The second race lasts 15 milliseconds. The record distance in this race is 40 millimeters.
>- The third race lasts 30 milliseconds. The record distance in this race is 200 millimeters.
>
>Your toy boat has a starting speed of zero millimeters per millisecond. For each whole millisecond you spend at the beginning of the race holding down the button, the boat's speed increases by one millimeter per millisecond.
>
>So, because the first race lasts 7 milliseconds, you only have a few options:

>- Don't hold the button at all (that is, hold it for 0 milliseconds) at the start of the race. The boat won't move; it will have traveled 0 millimeters by the end of the race.
>- Hold the button for 1 millisecond at the start of the race. Then, the boat will travel at a speed of 1 millimeter per millisecond for 6 milliseconds, reaching a total distance traveled of 6 millimeters.
>- Hold the button for 2 milliseconds, giving the boat a speed of 2 millimeters per millisecond. It will then get 5 milliseconds to move, reaching a total distance of 10 millimeters.
>- Hold the button for 3 milliseconds. After its remaining 4 milliseconds of travel time, the boat will have gone 12 millimeters.
>- Hold the button for 4 milliseconds. After its remaining 3 milliseconds of travel time, the boat will have gone 12 millimeters.
>- Hold the button for 5 milliseconds, causing the boat to travel a total of 10 millimeters.
>- Hold the button for 6 milliseconds, causing the boat to travel a total of 6 millimeters.
>- Hold the button for 7 milliseconds. That's the entire duration of the race. You never let go of the button. The boat can't move until you let go of the button. Please make sure you let go of the button so the boat gets to move. 0 millimeters.
>
>Since the current record for this race is 9 millimeters, there are actually 4 different ways you could win: you could hold the button for 2, 3, 4, or 5 milliseconds at the start of the race.
>
>In the second race, you could hold the button for at least 4 milliseconds and at most 11 milliseconds and beat the record, a total of 8 different ways to win.
>
>In the third race, you could hold the button for at least 11 milliseconds and no more than 19 milliseconds and still beat the record, a total of 9 ways you could win.
>
>To see how much margin of error you have, determine the number of ways you can beat the record in each race; in this example, if you multiply these values together, you get 288 (4 * 8 * 9).
>
>Determine the number of ways you could beat the record in each race. What do you get if you multiply these numbers together?

## Read input

Today's input is unusually simple but also unusual in that a single item is in columns rather than lines.

For both of the lines, I find all the numbers and store them in `times` and `distances` respectively.

In [1]:
import re
from utils import read_input

def transformer(line):
    return [int(v) for v in re.findall(r'\d+', line)]

times, records = read_input(6, transformer)

My first solution to this was to loop through every potential millisecond from 0 to `time` and then see how many of them are winning.

In [2]:
def first_solution(time, record):
    winning_strategies = 0
    for pressed_down in range(0, time+1):
        travelled = (time - pressed_down) * pressed_down
        if travelled > record:
            winning_strategies += 1
    return winning_strategies

The slight issue with this though is that when the numbers get large (as they did in part 2), there's a lot of unnecessary looping. The reason for that is that all the failing cases happen at the beginning and at the end.

So my second solution finds the lower threshold where we start winning and stops. It then starts from the end and finds the high threshold where we start winning and stops. And then combines these two together.

In [3]:
def find_winning_strategies(time, record):
    low_winner = 0
    high_winner = 0
    for pressed_down in range(0, time+1):
        travelled = (time - pressed_down) * pressed_down
        if travelled > record:
            low_winner = pressed_down
            break
    for pressed_down in range(time, 0, -1):
        travelled = (time - pressed_down) * pressed_down
        if travelled > record:
            high_winner = time - pressed_down
            break
    # We add +1 here because for a time of 7 ms, there are 8 attempts (one at 0ms)
    return (time + 1) - low_winner - high_winner

To get the final result, we combine the times and records with [zip](https://docs.python.org/3.3/library/functions.html#zip) that is a very handy function. It takes any number of iterables like lists or tuples and on each iteration, takes the nth item from each iterable.

```python
fruits = ['apple', 'banana', 'pear']
colors = ['red', 'yellow', 'green']
for fruit, color in zip(fruits, colors):
    print(f'{fruit} is {color}'
```

would print "apple is red", "banana is yellow", "pear is green"

In [4]:
part_1 = 1
for time, record in zip(times, records):
    part_1 *= find_winning_strategies(time, record)
print(f'Solution: {part_1}')
assert part_1 == 1731600

Solution: 1731600


## Part 2

>As the race is about to start, you realize the piece of paper with race times and record distances you got earlier actually just has very bad kerning. There's really only one race - ignore the spaces between the numbers on each line.
>
>So, the example from before:
>```
Time:      7  15   30
Distance:  9  40  200
>```
>...now instead means this:
>```
Time:      71530
Distance:  940200
>```
>
>Now, you have to figure out how many ways there are to win this single race. In this example, the race lasts for 71530 milliseconds and the record distance you need to beat is 940200 millimeters. You could hold the button anywhere from 14 to 71516 milliseconds and beat the record, a total of 71503 ways!
>
>How many ways can you beat the record in this one much longer race?

Them pesky elves with their incoherent writing again. All we need to do here is to fix the input by combining the numbers into a single string and then calling the function once more.

Even my `first_solution` was able to solve this in a relatively short time but the optimized solution is much better.

In [5]:
new_time = int(''.join(str(t) for t in times))
new_record = int(''.join(str(d) for d in records))

part_2 = find_winning_strategies(new_time, new_record)
print(f'Solution: {part_2}')
assert part_2 == 40087680

Solution: 40087680


## Two stars!

Today, we're at 11 out of 50 stars.